In [1]:
import pandas as pd
import urllib.request
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://press.un.org/en"

In [3]:
def is_press_release(soup):
    press_release_tag = soup.find('a', {'href': '/en/press-release', 'hreflang': 'en'})
    if press_release_tag:
        return True
    else:
        return False

In [19]:
def is_plenary_session(soup):
    tag = soup.find('span', class_="ep_name", string="Plenary session")
    if tag:
        return True
    else:
        return False


In [4]:
def contains_crisis(soup):
    if 'crisis' in soup.get_text().lower():
        return True
    else:
        return False

In [5]:
def add_link(link, visited, q):
    link_url = link['href']
    if link_url.startswith('/'):
        if q == 'Q1':
            link_url = 'https://press.un.org' + link_url
        elif q == 'Q2':
            link_url = 'https://www.europarl.europa.eu' + link_url
    if not link_url.startswith('#') and link_url not in visited:
        return link_url
    return None

In [10]:
import requests
from bs4 import BeautifulSoup

def find_press_release(url, visited, q):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException:
        return None, None, []

    press_release = None
    if q == 'Q1' and is_press_release(soup) and contains_crisis(soup):
        press_release = url
    elif q == 'Q2' and is_plenary_session(soup) and contains_crisis(soup):
        press_release = url

    links = []
    for link in soup.find_all('a', href=True):
        new_link = add_link(link, visited, q)
        if new_link:
            links.append(new_link)

    source_code = str(soup)  
    return press_release, source_code, links  

In [11]:
def get_press_releases(seed_url, q, print = False):
    visited = set()
    to_visit = [seed_url]
    press_releases = []
    source_codes = [] 
    max_ = 800
    now = 0
    while to_visit and len(press_releases) < 10 and now < max_:
        url = to_visit.pop(0)
        if url in visited:
            continue
        visited.add(url)
        if print == True:
            print(url)

        press_release, source_code, links = find_press_release(url, visited, q)  
        if press_release:
            press_releases.append(press_release)
            source_codes.append(source_code)  
        to_visit.extend(links)
        now += 1

    return press_releases, source_codes  

# Q1

In [12]:
urls_1, sc1 = get_press_releases('https://press.un.org/en' , 'Q1')

/Users/alex/.pyenv/versions/my-project/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [13]:
strs = []
for u in urls_1:
    print(u)
    strs += [u]


https://press.un.org/en/2023/sgsm21982.doc.htm
https://press.un.org/en/2023/sgsm21980.doc.htm
https://press.un.org/en/2023/sgsm21978.doc.htm
https://press.un.org/en/2023/sgsm21947.doc.htm
https://press.un.org/en/2023/dsgsm1874.doc.htm
https://press.un.org/en/2023/sgsm21952.doc.htm
https://press.un.org/en/2023/sgsm21876.doc.htm
https://press.un.org/en/2023/sgsm21852.doc.htm
https://press.un.org/en/2023/sgsm21806.doc.htm
https://press.un.org/en/2023/dsgsm1848.doc.htm


# Q2

In [33]:
urls_2 = get_press_releases('https://www.europarl.europa.eu/news/en/press-room', 'Q2')

In [40]:
for u in urls_2:
    print(u)

https://www.europarl.europa.eu/news/en/press-room/20230929IPR06132/nagorno-karabakh-meps-demand-review-of-eu-relations-with-azerbaijan


# Q2 After ED 

In [14]:
url = 'https://www.europarl.europa.eu/news/en/press-room/page/0'

In [15]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [20]:

viable_link = []
sc2 = []  
for i in range(10):
    url = 'https://www.europarl.europa.eu/news/en/press-room/page/'
    url += str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    if len(viable_link) == 10:
        break
    for item in soup.find_all('a', href=True):
        if 'https://www.europarl.europa.eu/news/en/press-room/20' in item['href']:
            url = item['href']
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            if is_plenary_session(soup) and contains_crisis(soup):
                print(url)
                viable_link.append(url)
                sc2.append(str(soup))

https://www.europarl.europa.eu/news/en/press-room/20230929IPR06132/nagorno-karabakh-meps-demand-review-of-eu-relations-with-azerbaijan
https://www.europarl.europa.eu/news/en/press-room/20230929IPR06130/parliament-argues-for-a-top-up-to-multi-annual-budget-for-crisis-response
https://www.europarl.europa.eu/news/en/press-room/20230911IPR04923/reduce-demand-and-protect-people-in-prostitution-say-meps
https://www.europarl.europa.eu/news/en/press-room/20230911IPR04918/svietlana-tsikhanouskaya-to-meps-support-belarusians-european-aspirations
https://www.europarl.europa.eu/news/en/press-room/20230911IPR04908/meps-vote-to-strengthen-eu-defence-industry-through-common-procurement
https://www.europarl.europa.eu/news/en/press-room/20230707IPR02427/covid-19-parliament-adopts-roadmap-to-better-prepare-for-future-health-crises
https://www.europarl.europa.eu/news/en/press-room/20230707IPR02421/parliament-adopts-new-rules-to-boost-energy-savings
https://www.europarl.europa.eu/news/en/press-room/202307

In [17]:
for i in range(1,11):
    txts = '1_'
    txts += str(i)
    txts += '.txt'
    with open(txts, "w") as file:
        file.write(sc1[i-1])


In [21]:
for i in range(1,11):
    txts = '2_'
    txts += str(i)
    txts += '.txt'
    with open(txts, "w") as file:
        file.write(sc2[i-1])
        
